In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [2]:
import pickle
import pandas as pd
import pyarrow

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [7]:
y_pred.mean()

16.191691679979066

In [8]:
year = 2021
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [9]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration,ride_id
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173,82,None,B00021,10.666667,2021/02_1
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173,56,None,B00021,14.566667,2021/02_2
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82,129,None,B00021,7.950000,2021/02_3
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1,225,None,B00037,13.800000,2021/02_4
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1,61,None,B00037,8.966667,2021/02_5
...,...,...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,-1,31,None,B01717,13.533333,2021/02_1037687
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,-1,169,None,B01717,11.466667,2021/02_1037688
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28,171,None,B03285,25.383333,2021/02_1037689
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16,252,None,B03285,18.050000,2021/02_1037690


In [13]:
df_result = pd.DataFrame(y_pred, columns = ['Prediction'])
df_result['ride_id'] = df['ride_id']
df_result

,Prediction,ride_id
0,14.539865,NaN
1,13.740422,2021/02_1
2,15.593339,2021/02_2
3,15.188118,2021/02_3
4,13.817206,2021/02_4
...,...,...
990108,12.433246,2021/02_990108
990109,12.910885,2021/02_990109
990110,15.834923,2021/02_990110
990111,16.783176,2021/02_990111


In [14]:
df_result.to_parquet(
    'OutputPred.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)